In [1]:
from PIL import Image
import numpy as np

In [2]:
image = Image.open('black cock.jpg')

In [3]:
image_array = np.array(image)
print(image_array)

[[[ 61  63  42]
  [ 61  63  42]
  [ 61  63  42]
  ...
  [148 201 147]
  [148 201 149]
  [148 200 151]]

 [[ 62  64  43]
  [ 62  64  43]
  [ 62  64  43]
  ...
  [148 201 147]
  [149 202 150]
  [149 201 152]]

 [[ 65  66  48]
  [ 65  66  48]
  [ 64  65  47]
  ...
  [148 201 147]
  [149 202 148]
  [149 202 150]]

 ...

 [[133 110  94]
  [126 103  87]
  [123 100  86]
  ...
  [175 205 135]
  [196 222 159]
  [231 239 202]]

 [[123  97  82]
  [110  84  69]
  [101  75  62]
  ...
  [190 217 148]
  [199 223 163]
  [200 206 172]]

 [[108  82  67]
  [107  81  66]
  [ 98  72  59]
  ...
  [164 191 124]
  [179 203 143]
  [165 170 138]]]


In [32]:
red_channel = image_array[:,:,0]
green_channel = image_array[:,:,1]
blue_channel = image_array[:,:,2]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed